In [29]:
import spacy
import mysql.connector
from openai import OpenAI

# load env api keys
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("No API key found")

In [39]:
class QueryParser:
    def __init__(self, text):
        self.text = text
        self.nlp = spacy.load("en_core_web_sm")
        self.doc = self.nlp(text)
        self.car_brands = [
            "Abarth",
            "Alfa Romeo",
            "Aston Martin",
            "Audi",
            "Bentley",
            "BMW",
            "Bugatti",
            "Cadillac",
            "Chevrolet",
            "Chrysler",
            "Citroën",
            "Dacia",
            "Daewoo",
            "Daihatsu",
            "Dodge",
            "Donkervoort",
            "DS",
            "Ferrari",
            "Fiat",
            "Fisker",
            "Ford",
            "Honda",
            "Hummer",
            "Hyundai",
            "Infiniti",
            "Iveco",
            "Jaguar",
            "Jeep",
            "Kia",
            "KTM",
            "Lada",
            "Lamborghini",
            "Lancia",
            "Land Rover",
            "Landwind",
            "Lexus",
            "Lotus",
            "Maserati",
            "Maybach",
            "Mazda",
            "McLaren",
            "Mercedes-Benz",
            "Mercedes",
            "MG",
            "Mini",
            "Mitsubishi",
            "Morgan",
            "Nissan",
            "Opel",
            "Peugeot",
            "Porsche",
            "Renault",
            "Rolls-Royce",
            "Rover",
            "Saab",
            "Seat",
            "Skoda",
            "Smart",
            "SsangYong",
            "Subaru",
            "Suzuki",
            "Tesla",
            "Toyota",
            "Volkswagen",
            "Volvo",
        ]
        self.fuel_types = ["Petrol", "Diesel", "Electric", "Hybrid", "EV", "PULP 95"]
        self.negation = ["not", "no", "without", "exclude", "excluding"]
        self.greater = [
            "above",
            "more than",
            "greater than",
            "higher than",
            "over",
            "exceeding",
        ]
        self.lesser = [
            "below",
            "less than",
            "lower than",
            "under",
            "beneath",
            "not exceeding",
        ]
        self.equal = [
            "equal to",
            "same as",
            "equal",
            "same",
            "around",
            "approximately",
            "about",
            "near",
            "nearly",
            "close to",
            "just over",
            "just under",
            "just above",
            "just below",
        ]
        self.query_information = {
            "brand": None,
            "model": None,
            "production_year": None,
            "price_min": None,
            "price_max": None,
            "price_avg": None,
            "excluded_brands": [],
            "fuel_type": None,
            "excluded_fuel_types": [],
        }
        self.money_signs = ["$", "€", "£", "₪"]
        self.money_names = ["dollar", "euro", "pound", "shekel"]
        self.parse_query()

    def parse_query(self):
        for sent in self.doc.sents:
            for token in sent:
                if token.text.lower() in self.negation:
                    context = self.doc[
                        token.i : min(len(self.doc), token.i + 5)
                    ].text.lower()
                    for brand in self.car_brands:
                        if brand.lower() in context:
                            self.query_information["excluded_brands"].append(brand)
                    for fuel in self.fuel_types:
                        if fuel.lower() in context:
                            self.query_information["excluded_fuel_types"].append(fuel)

                if token.text.lower() in [brand.lower() for brand in self.car_brands]:
                    if token.text.lower() not in [
                        excluded.lower()
                        for excluded in self.query_information["excluded_brands"]
                    ]:
                        self.query_information["brand"] = token.text
                        next_token = token.nbor()
                        if next_token.is_alpha or (
                            not next_token.is_alpha and not next_token.like_num
                        ):
                            self.query_information["model"] = next_token.text
                            if (
                                next_token.nbor().like_num
                                and len(next_token.nbor().text) == 4
                            ):
                                self.query_information["production_year"] = (
                                    next_token.nbor().text
                                )
                        elif next_token.like_num and len(next_token.text) == 4:
                            self.query_information["production_year"] = next_token.text
                        else:
                            self.query_information["model"] = None

                if token.text.lower() in [
                    fuel.lower() for fuel in self.fuel_types
                ] and token.text.lower() not in [
                    excluded.lower()
                    for excluded in self.query_information["excluded_fuel_types"]
                ]:
                    self.query_information["fuel_type"] = token.text

                if token.like_num:
                    context = self.doc[max(0, token.i - 5) : token.i].text.lower()
                    for keyword in self.greater:
                        if keyword in context:
                            self.query_information["price_min"] = int(token.text.replace(",", ""))
                    for keyword in self.lesser:
                        if keyword in context:
                            self.query_information["price_max"] = int(token.text.replace(",", ""))
                    for keyword in self.equal:
                        if keyword in context:
                            self.query_information["price_avg"] = int(token.text.replace(",", ""))

    def get_query_information(self):
        return self.query_information

In [31]:
text = "I want to purchase a new car, a Hyundai 2012 and not a Tesla and not a Toyota or Volvo, I want the car to be Hybrid aswell. I'm looking for a price more than 20,000$ but less than 30,000$, around 25,000$ please no petrol"
parser = QueryParser(text)
print("Query: ", text)
info = parser.get_query_information()
print("Query Information", parser.get_query_information())

Query:  I want to purchase a new car, a Hyundai 2012 and not a Tesla and not a Toyota or Volvo, I want the car to be Hybrid aswell. I'm looking for a price more than 20,000$ but less than 30,000$, around 25,000$ please no petrol
Query Information {'brand': 'Hyundai', 'model': None, 'production_year': '2012', 'price_min': 20000, 'price_max': 30000, 'price_avg': 25000, 'excluded_brands': ['Tesla', 'Toyota', 'Volvo'], 'fuel_type': 'Hybrid', 'excluded_fuel_types': ['Petrol']}


In [44]:
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "rag",
}

try:
    print("connecting...")
    # connecting to database using config
    conn = mysql.connector.connect(**db_config)
    print("connected")
    # creating a cursor object using the cursor() method, which is used to execute SQL queries
    cursor = conn.cursor()
    
    # building the query
    query = "SELECT * FROM vehicles WHERE "
    if info['brand']:
        query += f"brand = '{info['brand']}'"
    if info['model']:
        query += f" AND model = '{info['model']}'"
    if info['production_year']:
        query += f" AND prodyear = '{info['production_year']}'"
    if info['fuel_type']:
        query += f" AND fuel = '{info['fuel_type']}'"
    if info['price_avg']:
        query += f" AND price BETWEEN {info['price_avg'] - 2000} AND {info['price_avg'] + 2000}"
    else:
        if info['price_min']:
            query += f" AND price > {info['price_min']}"
        if info['price_max']:
            query += f" AND price < {info['price_max']}"
    # if info['excluded_brands']:
    #     query += " AND brand NOT IN ("
    #     for brand in info['excluded_brands']:
    #         query += f"'{brand}',"
    #     query = query[:-1] + ")"
    # if info['excluded_fuel_types']:
    #     query += " AND fuel_type NOT IN ("
    #     for fuel in info['excluded_fuel_types']:
    #         query += f"'{fuel}',"
    #     query = query[:-1] + ")"
    print(query)
    cursor.execute(query)
    result = cursor.fetchall()
    print(result)
except mysql.connector.Error as e:
    print(f"Error connecting to MySQL Platform: {e}")
finally: # closing database connection & cursor
    cursor.close()
    conn.close()


connecting...
connected
SELECT * FROM vehicles WHERE brand = 'Hyundai' AND prodyear = '2012' AND fuel = 'Hybrid' AND price BETWEEN 23000 AND 27000
[(1002, 'Hyundai', 'i20', 2012, 25000, 'Hybrid'), (1003, 'Hyundai', 'i30', 2012, 24000, 'Hybrid'), (1004, 'Hyundai', 'Sedan', 2012, 26500, 'Hybrid')]


#### Once we have retrieved a range of options from our database we will build a query for chat GPT or any other AI

In [47]:
prompt = "I have the following vehicular options for you: \n"
for option in result:
    prompt += f"{option}\n"
prompt += "Which one of those would you say is the best option wise? Fuel economy, price, comfort and etc with all factors considered?"
prompt += " I need you to write me a simple single line of response which only includes the vehicle and it's details. For example: 'Hyundai i20 2012'. Make sure you include the year of the model in your response"

print(prompt)

I have the following vehicular options for you: 
(1002, 'Hyundai', 'i20', 2012, 25000, 'Hybrid')
(1003, 'Hyundai', 'i30', 2012, 24000, 'Hybrid')
(1004, 'Hyundai', 'Sedan', 2012, 26500, 'Hybrid')
Which one of those would you say is the best option wise? Fuel economy, price, comfort and etc with all factors considered? I need you to write me a simple single line of response which only includes the vehicle and it's details. For example: 'Hyundai i20 2012'. Make sure you include the year of the model in your response


#### Next step would be connecting to an AI API to send forward the query and return the result to the user

In [49]:
client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a vehicle recommendation assistant. Respond concisely and provide only the vehicle information per the user's request."},
        {
            "role": "user",
            "content": prompt
        }
    ]
)

print(completion.choices[0].message.content)

Hyundai i30 2012


In [35]:
# import mysql.connector
# import csv

# try:
#     conn = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="",  # Leave it empty if there's no password for root
#         database="rag",
#     )
#     cursor = conn.cursor()
#     cursor.execute("SELECT MAX(id) FROM vehicles")
#     max_id = cursor.fetchone()[0] or 0

#     with open("vehicles.csv", "r") as file:
#         reader = csv.DictReader(file)
#         insert_query = """
#         INSERT INTO vehicles (id, brand, model, prodyear, price, fuel)
#         VALUES (%s, %s, %s, %s, %s, %s)
#         """
#         for row in reader:
#             max_id += 1
#             cursor.execute(
#                 insert_query,
#                 (
#                     max_id,
#                     row["brand"],
#                     row["model"],
#                     int(row["prodyear"]),
#                     int(row["price"]),
#                     row["fuel"],
#                 ),
#             )
#     conn.commit()
#     print("Data inserted successfully.")
# except mysql.connector.Error as err:
#     print(f"Error: {err}")
# finally:
#     cursor.close()
#     conn.close()
#     print("DB connection closed.")